# 1. Getting the data

### The main code for querying the Places API
After running this the object response will have the json we queried stored in pure text format.\
Some adjustments will be needed to get more than 20 items from the query, we can do this by using nextPageToken.

In [12]:
import requests, json
from secretfile import api_key

origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = 1000
query_string = "burger"

url_textsearch = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_string}&type=restaurant&location={origin_location}&radius={radius_in_meters}&key={api_key}"

payload={}
headers = {}

def get_request(url):
    response = requests.request("GET", url, headers=headers, data=payload)
    response.raise_for_status()
    return response

response = get_request(url_textsearch)


### Formatting the data we just got

We want to write our results to a file temporarily, then populate a dataframe with the data

In [20]:
import pandas as pd

In [13]:
with open('restaurants.json', 'w' , encoding='utf-8') as w:
    w.write(response.text)

There is still some data that we have not been able to catch; the opening hours\
We will need to iterate through the names in a Place Details query against the Places API

In [35]:
# The keys function as attributes in this dict
result_dict = {"place_id": [],  "name": [], "rating": [], "user_ratings_total": [], "formatted_address": [], "opening_hours": []}

# Populate the dict with values from the results
with open('restaurants.json', 'r', encoding='utf-8') as data_file:
    data = json.load(data_file)
    for restaurant in data['results']:
        result_dict['place_id'].append(restaurant['place_id'])
        result_dict['name'].append(restaurant['name'])
        result_dict['rating'].append(restaurant['rating'])
        result_dict['user_ratings_total'].append(restaurant['user_ratings_total'])
        result_dict['formatted_address'].append(restaurant['formatted_address'])


origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = "1000"
query_string = "burger"

        
# We query for the opening hours of every restaurant
for place_id in result_dict['place_id']:
    url_findplace = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,opening_hours&key={api_key}"
    response = get_request(url_findplace)
    print(response.text)
    data = json.load(response.text)
    print()
    
    

{
   "html_attributions" : [],
   "result" : {
      "name" : "Stockyard Burger Joint",
      "opening_hours" : {
         "open_now" : false,
         "periods" : [
            {
               "close" : {
                  "day" : 0,
                  "time" : "2300"
               },
               "open" : {
                  "day" : 0,
                  "time" : "1200"
               }
            },
            {
               "close" : {
                  "day" : 2,
                  "time" : "0000"
               },
               "open" : {
                  "day" : 1,
                  "time" : "1130"
               }
            },
            {
               "close" : {
                  "day" : 3,
                  "time" : "0000"
               },
               "open" : {
                  "day" : 2,
                  "time" : "1130"
               }
            },
            {
               "close" : {
                  "day" : 4,
                  "time" : "0000"
 

AttributeError: 'str' object has no attribute 'read'

In [33]:
# Used this to test the Places details API

place_id = "ChIJN1t_tDeuEmsRUsoyG83frY4"
url_findplace = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,opening_hours&key={api_key}"
response = get_request(url_findplace)
print(response.text)

{
   "html_attributions" : [],
   "result" : {
      "name" : "Google Workplace 6",
      "opening_hours" : {
         "open_now" : false,
         "periods" : [
            {
               "close" : {
                  "day" : 1,
                  "time" : "1700"
               },
               "open" : {
                  "day" : 1,
                  "time" : "0900"
               }
            },
            {
               "close" : {
                  "day" : 2,
                  "time" : "1700"
               },
               "open" : {
                  "day" : 2,
                  "time" : "0900"
               }
            },
            {
               "close" : {
                  "day" : 3,
                  "time" : "1700"
               },
               "open" : {
                  "day" : 3,
                  "time" : "0900"
               }
            },
            {
               "close" : {
                  "day" : 4,
                  "time" : "1700"
     

Now that we have the data we need to put it in a dataframe


In [19]:

# Create a neat dataframe and sort the rows by rating, user_ratings_total
df = pd.DataFrame.from_dict(result_dict)
df['rating'] = pd.to_numeric(df['rating'])
df['user_ratings_total'] = pd.to_numeric(df['user_ratings_total'])
df.sort_values(['rating', 'user_ratings_total'], ascending=False)


,name,rating,user_ratings_total,formatted_address
9,"DINÉ Burgers, femman huset",4.7,30,"Postgatan 26-31, 411 06 Göteborg"
17,The Barn,4.5,2394,"Kyrkogatan 11, 411 15 Göteborg"
7,2112,4.5,1914,"Magasinsgatan 5, 411 18 Göteborg"
11,Master Burger,4.5,53,"Södra Vägen 30, 412 54 Göteborg"
10,Johnny's Burger and Shakes,4.5,35,"Vasa Allen / avenue, Vasagatan 56, 411 36 Göte..."
1,DINÉ Burgers,4.4,927,"Drottninggatan 31, 411 14 Göteborg"
2,Tugg,4.3,2344,"Drottninggatan 21, 411 14 Göteborg"
0,Stockyard Burger Joint,4.3,889,"Storgatan 51, 411 38 Göteborg"
13,Butcher's Market,4.3,827,"Storgatan 47, 411 38 Göteborg"
16,The Goods Basement,4.3,120,"Lorensbergsgatan 6, 411 36 Göteborg"


### Storing the data